<a href="https://colab.research.google.com/github/MuhammadFasihArif/Audio-Deep-fake-detector/blob/main/audio_df_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/noorchauhan/DeepFake-Audio-Detection-MFCC.git
!cd deepfake-audio-detection

Cloning into 'DeepFake-Audio-Detection-MFCC'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (11/11), done.
remote: Total 50 (delta 10), reused 10 (delta 10), pack-reused 39 (from 1)
Receiving objects: 100% (50/50), 13.87 MiB | 23.28 MiB/s, done.
Resolving deltas: 100% (18/18), done.
/bin/bash: line 1: cd: deepfake-audio-detection: No such file or directory


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohammedabdeldayem/the-fake-or-real-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/the-fake-or-real-dataset


In [ ]:
import os
import glob
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib

def extract_mfcc_features(audio_path, n_mfcc=13, n_fft=2048, hop_length=512):
    try:
        audio_data, sr = librosa.load(audio_path, sr=None)
    except Exception as e:
        print(f"Error loading audio file {audio_path}: {e}")
        return None

    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    return np.mean(mfccs.T, axis=0)

def create_dataset(directory, label):
    X, y = [], []
    audio_files = glob.glob(os.path.join(directory, "*.wav"))
    for audio_path in audio_files:
        mfcc_features = extract_mfcc_features(audio_path)
        if mfcc_features is not None:
            X.append(mfcc_features)
            y.append(label)
        else:
            print(f"Skipping audio file {audio_path}")

    print("Number of samples in", directory, ":", len(X))
    print("Filenames in", directory, ":", [os.path.basename(path) for path in audio_files])
    return X, y


def train_model(X, y):
    unique_classes = np.unique(y)
    print("Unique classes in y_train:", unique_classes)

    if len(unique_classes) < 2:
        raise ValueError("Atleast 2 set is required to train")

    print("Size of X:", X.shape)
    print("Size of y:", y.shape)

    class_counts = np.bincount(y)
    if np.min(class_counts) < 2:
        print("Combining both classes into one for training")
        X_train, y_train = X, y
        X_test, y_test = None, None
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

        print("Size of X_train:", X_train.shape)
        print("Size of X_test:", X_test.shape)
        print("Size of y_train:", y_train.shape)
        print("Size of y_test:", y_test.shape)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    if X_test is not None:
        X_test_scaled = scaler.transform(X_test)

        svm_classifier = SVC(kernel='linear', random_state=42)
        svm_classifier.fit(X_train_scaled, y_train)

        y_pred = svm_classifier.predict(X_test_scaled)

        accuracy = accuracy_score(y_test, y_pred)
        confusion_mtx = confusion_matrix(y_test, y_pred)

        print("Accuracy:", accuracy)
        print("Confusion Matrix:")
        print(confusion_mtx)
    else:
        print("Insufficient samples for stratified splitting. Combine both classes into one for training.")
        print("Train on all available data.")

        svm_classifier = SVC(kernel='linear', random_state=42)
        svm_classifier.fit(X_train_scaled, y_train)

    # Save the trained SVM model and scaler
    model_filename = "svm_model.pkl"
    scaler_filename = "scaler.pkl"
    joblib.dump(svm_classifier, model_filename)
    joblib.dump(scaler, scaler_filename)

def analyze_audio(input_audio_path):
    model_filename = "svm_model.pkl"
    scaler_filename = "scaler.pkl"
    svm_classifier = joblib.load(model_filename)
    scaler = joblib.load(scaler_filename)

    if not os.path.exists(input_audio_path):
        print("Error: The specified file does not exist.")
        return
    elif not input_audio_path.lower().endswith(".wav"):
        print("Error: The specified file is not a .wav file.")
        return

    mfcc_features = extract_mfcc_features(input_audio_path)

    if mfcc_features is not None:
        mfcc_features_scaled = scaler.transform(mfcc_features.reshape(1, -1))
        prediction = svm_classifier.predict(mfcc_features_scaled)
        if prediction[0] == 0:
            print("The input audio is classified as genuine.")
        else:
            print("The input audio is classified as deepfake.")
    else:
        print("Error: Unable to process the input audio.")

def main():
    genuine_dir = r"/kaggle/input/the-fake-or-real-dataset/for-original/for-original/testing/real"
    deepfake_dir = r"/kaggle/input/the-fake-or-real-dataset/for-original/for-original/testing/fake"

    X_genuine, y_genuine = create_dataset(genuine_dir, label=0)
    X_deepfake, y_deepfake = create_dataset(deepfake_dir, label=1)

    # Check if each class has at least two samples
    if len(X_genuine) < 2 or len(X_deepfake) < 2:
        print("Each class should have at least two samples for stratified splitting.")
        print("Combining both classes into one for training.")
        X = np.vstack((X_genuine, X_deepfake))
        y = np.hstack((y_genuine, y_deepfake))
    else:
        X = np.vstack((X_genuine, X_deepfake))
        y = np.hstack((y_genuine, y_deepfake))

    train_model(X, y)

if __name__ == "__main__":
    main()

    user_input_file = input("Enter the path of the .wav file to analyze: ")
    analyze_audio(user_input_file)


Number of samples in /kaggle/input/the-fake-or-real-dataset/for-original/for-original/testing/real : 2264
Filenames in /kaggle/input/the-fake-or-real-dataset/for-original/for-original/testing/real : ['file477.wav', 'file1563.wav', 'file116.wav', 'file126.wav', 'file884.wav', 'file1733.wav', 'file931.wav', 'file2047.wav', 'file2037.wav', 'file303.wav', 'file501.wav', 'file5.wav', 'file1397.wav', 'file1656.wav', 'file1185.wav', 'file250.wav', 'file1646.wav', 'file648.wav', 'file563.wav', 'file1892.wav', 'file1723.wav', 'file1315.wav', 'file1959.wav', 'file12.wav', 'file1913.wav', 'file548.wav', 'file757.wav', 'file737.wav', 'file534.wav', 'file1497.wav', 'file1996.wav', 'file1296.wav', 'file1899.wav', 'file1350.wav', 'file478.wav', 'file1997.wav', 'file1511.wav', 'file776.wav', 'file284.wav', 'file136.wav', 'file1391.wav', 'file1503.wav', 'file287.wav', 'file1791.wav', 'file34.wav', 'file1804.wav', 'file874.wav', 'file564.wav', 'file2122.wav', 'file635.wav', 'file2028.wav', 'file1918.wav